In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('usd_curve.csv')
times = data['t'].values
df = data['df'].values

class HullWhite():
    def __init__(self, a, sigma, timesteps, x0=0):
        self.a = a
        self.sigma = sigma
        self.timesteps = timesteps
        self.x0 = x0
        
    def hw_exp(self, x, t1, t2):
        return x * np.exp(-self.a * (t2 - t1))
    
    def hw_var(self, t1, t2):
        return self.sigma ** 2 * (1 - np.exp(-2 * self.a * (t2 - t1))) / (2 * self.a)

    def MC(self, sims):
        x = np.zeros((sims, self.timesteps))
        for i in range(sims):
            x[i,0] = self.x0
            for j in range(1, self.timesteps):
                Z1 = np.random.normal(0,1)
                x[i,j] = self.hw_exp(x[i,j-1], j-1, j) + Z1 * (self.hw(j-1,j) ** 0.5)
        return x     
    


In [3]:
class DiscountFactors():
    def __init__(self, a, sigma):
        self.a = a
        self.sigma = sigma
        
    def q_t(self, t, T):
         return (1 - np.exp(-self.a * (T - t))) / self.a

    def cov(self, t_i, t_j):
        return self.sigma ** 2 * (np.exp(-self.a * abs(t_j - t_i)) - np.exp(-self.a * (t_j + t_i))) / (2 * self.a)
 
    def hw_phi(self, q_val, cov_val):
        return sum(q * cov for q, cov in zip(q_values, cov_values))

    def hw_gamma(self, tn):
        return 0.5 * self.cov(tn, tn)

    def interpolated_df(self, t, time, df):
        return np.interp(t, time, df)
    
    def df(self, t_n, T, x_tn, q_val, cov_val):
        B_tn = self.interpolated_df(t_n, times, df)
        B_T = self.interpolated_df(T, times, df)
        if t_n == 0:
            discount_factor = B_T / B_tn 
        else:
            phi = self.hw_phi(q_val, cov_val)
            gamma = self.hw_gamma(t_n)
            q_tn = self.q_t(t_n, T)
            discount_factor = (B_T / B_tn) * np.exp(-(x_tn + phi) * q_tn - gamma * (q_tn ** 2))
        return discount_factor
        

In [ ]:
# build discount factors 